# Tag Prediction using Multi-Label Text Classification


# Link drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install scikit-multilearn

     |████████████████████████████████| 89 kB 4.0 MB/s 


# Code

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# The questions
ENCODING = 'ISO-8859-1'

df_q = pd.read_csv("drive/My Drive/B2_NLP_Team_9/dataset/Questions.csv", encoding=ENCODING)
df_q.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [ ]:
# the tags

df_t = pd.read_csv("drive/My Drive/B2_NLP_Team_9/dataset/Tags.csv", encoding=ENCODING)
df_t.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [ ]:
df_t['Tag'] = df_t['Tag'].astype(str)

# group all tags given to same question into a single string
grouped_tags = df_t.groupby('Id')['Tag'].apply(lambda tags: ' '.join(tags))

grouped_tags.head()

Id
80                            flex actionscript-3 air
90       svn tortoisesvn branch branching-and-merging
120                               sql asp.net sitemap
180    algorithm language-agnostic colors color-space
260           c# .net scripting compiler-construction
Name: Tag, dtype: object

In [ ]:
# reset index for simplicity
grouped_tags.reset_index()

df_tags_final = pd.DataFrame({'Id': grouped_tags.index, 'Tags': grouped_tags.values})

df_tags_final.head()

,Id,Tags
0,80,flex actionscript-3 air
1,90,svn tortoisesvn branch branching-and-merging
2,120,sql asp.net sitemap
3,180,algorithm language-agnostic colors color-space
4,260,c# .net scripting compiler-construction


In [ ]:
# Drop unnecessary columns from questions
df_q.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)

In [ ]:
# Merge questions and tags into a single dataframe
df = df_q.merge(df_tags_final, on='Id')

del df_q
del df_t

df.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex actionscript-3 air
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn tortoisesvn branch branching-and-merging
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql asp.net sitemap
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm language-agnostic colors color-space
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c# .net scripting compiler-construction


In [ ]:
# remove questions with score lower than 5
df = df[df['Score'] > 5]

print(df.shape)

(72950, 5)


In [ ]:
# split tags into list
df['Tags'] = df['Tags'].apply(lambda tags: tags.lower().split())

df.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"


In [ ]:
# get all tags in the dataset
all_tags = []

for tags in df['Tags'].values:
    for tag in tags:
        all_tags.append(tag)

# print(all_tags)

In [ ]:
import nltk

# create a frequency list of the tags
tag_freq = nltk.FreqDist(list(all_tags))

# get most common tags
tag_freq.most_common(25)

[('c#', 6907),
 ('java', 6862),
 ('javascript', 5560),
 ('android', 5052),
 ('python', 4442),
 ('c++', 4369),
 ('php', 3024),
 ('jquery', 2770),
 ('.net', 2705),
 ('ios', 2685),
 ('html', 2129),
 ('css', 1980),
 ('c', 1845),
 ('iphone', 1781),
 ('objective-c', 1768),
 ('ruby-on-rails', 1524),
 ('sql', 1389),
 ('asp.net', 1302),
 ('mysql', 1286),
 ('ruby', 1249),
 ('r', 1120),
 ('git', 1013),
 ('asp.net-mvc', 1004),
 ('linux', 935),
 ('sql-server', 907)]

In [ ]:
# get the most common 20 tags without the count
tag_features = list(map(lambda x: x[0], tag_freq.most_common(20)))

print(tag_features)

['c#', 'java', 'javascript', 'android', 'python', 'c++', 'php', 'jquery', '.net', 'ios', 'html', 'css', 'c', 'iphone', 'objective-c', 'ruby-on-rails', 'sql', 'asp.net', 'mysql', 'ruby']


In [ ]:
# Filter the tags from the dataset and remove all tags that does not belong to the tag_features
def keep_common(tags):

    filtered_tags = []

    # filter tags
    for tag in tags:
        if tag in tag_features:
            filtered_tags.append(tag)

    # return the filtered tag list
    return filtered_tags

# apply the function to filter in dataset
df['Tags'] = df['Tags'].apply(lambda tags: keep_common(tags))

# set the Tags column as None for those that do not have a most common tag
df['Tags'] = df['Tags'].apply(lambda tags: tags if len(tags) > 0 else None)

# Now we will drop all the columns that contain None in Tags column
df.dropna(subset=['Tags'], inplace=True)

df.shape

(48976, 5)

In [ ]:
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.snowball import SnowballStemmer

tokenizer = ToktokTokenizer()
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

# Preprocess the text for vectorization
# - Remove HTML
# - Remove stopwords
# - Remove special characters
# - Convert to lowercase
# - Stemming

def remove_html(text):
    # Remove html and convert to lowercase
    return re.sub(r"\<[^\>]\>", "", text).lower()

def remove_stopwords(text):
    # tokenize the text
    words = tokenizer.tokenize(text)

    filtered = [w for w in words if not w in stop_words]
    return ' '.join(map(str, filtered))

def remove_punc(text):
    #tokenize
    tokens = tokenizer.tokenize(text)

    # remove punctuations from each token
    tokens = list(map(lambda token: re.sub(r"[^A-Za-z0-9]+", " ", token).strip(), tokens))

    # remove empty strings from tokens
    tokens = list(filter(lambda token: token, tokens))

    return ' '.join(map(str, tokens))

def stem_text(text):
    #tokenize
    tokens = tokenizer.tokenize(text)

    # stem each token
    tokens = list(map(lambda token: stemmer.stem(token), tokens))

    return ' '.join(map(str, tokens))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# drop Id and Score columns since we don't need them
df.drop(columns=['Id', 'Score'], inplace=True)

df.head()

,Title,Body,Tags
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"
5,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[c++]
6,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,[.net]
8,Automatically update version number,<p>I would like the version property of my app...,[c#]


In [ ]:
# apply preprocessing to title and bodya
df['Title'] = df['Title'].apply(lambda x: remove_html(x))
df['Title'] = df['Title'].apply(lambda x: remove_stopwords(x))
df['Title'] = df['Title'].apply(lambda x: remove_punc(x))
df['Title'] = df['Title'].apply(lambda x: stem_text(x))

df.head()

,Title,Body,Tags
2,asp net site map,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
4,ad script function net applic,<p>I have a little game written in C#. It uses...,"[c#, .net]"
5,use nest class case,<p>I am working on a collection of classes use...,[c++]
6,homegrown consumpt web servic,<p>I've been writing a few web services for a ...,[.net]
8,automat updat version number,<p>I would like the version property of my app...,[c#]


In [ ]:
# apply preprocessing to title and body
df['Body'] = df['Body'].apply(lambda x: remove_html(x))
df['Body'] = df['Body'].apply(lambda x: remove_stopwords(x))
df['Body'] = df['Body'].apply(lambda x: remove_punc(x))
df['Body'] = df['Body'].apply(lambda x: stem_text(x))

df.head()

,Title,Body,Tags
2,asp net site map,anyon got experi creat strong sql base asp net...,"[sql, asp.net]"
4,ad script function net applic,littl game written c use databas back end a hr...,"[c#, .net]"
5,use nest class case,work collect class use video playback record o...,[c++]
6,homegrown consumpt web servic,write web servic net app readi consum them see...,[.net]
8,automat updat version number,would like version properti applic increment b...,[c#]


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

In [ ]:
X_title = df['Title']
X_body = df['Body']
y = df['Tags']

del df

# binarize our tags
binarizer = MultiLabelBinarizer()
y_bin = binarizer.fit_transform(y)

In [ ]:
# vectorize
vectorizer_title = TfidfVectorizer(
    analyzer = 'word',
    strip_accents = None,
    encoding = 'utf-8',
    preprocessor=None,
    max_features=10000)

vectorizer_body = TfidfVectorizer(
    analyzer = 'word',
    strip_accents = None,
    encoding = 'utf-8',
    preprocessor=None,
    max_features=10000)

X_title_vect = vectorizer_title.fit_transform(X_title)
X_body_vect = vectorizer_body.fit_transform(X_body)

X = hstack([X_title_vect, X_body_vect])

In [ ]:
X.shape

(48976, 20000)

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size = 0.2, random_state = 0)

del X

In [ ]:
def j_score(y_true, y_pred):
  jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
  return jaccard.mean()

def print_score(y_test, y_pred):
  # calculate Jacard score
  print('Jacard score: {}'.format(j_score(y_test, y_pred.toarray())))
  print('----')
  # # calculate Accuracy
  # print("Accuracy:", accuracy_score(y_test, y_pred))
  # print('----')
  # calculate recall
  print("Recall:", recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
  print('----')
  # calculate precision
  print("Precision: ", precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
  print('----')
  # calculate hamming loss
  print("Hamming Loss (%): ", hamming_loss(y_pred, y_test)*100)
  print('----')
  # calculate F1 score
  print("F1 Score: ", f1_score(y_pred, y_test, average='weighted'))
  print('----')

In [ ]:
# Develop the model
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import BinaryRelevance # gives better precision

svc = LinearSVC()
clf = BinaryRelevance(svc)

# fit training data
clf.fit(X_train, y_train)


BinaryRelevance(classifier=LinearSVC(), require_dense=[True, True])

In [ ]:
# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, hamming_loss, f1_score

# make prediction
y_pred = clf.predict(X_test)

print_score(y_test, y_pred)

Jacard score: 0.6670273581053491
----
Recall: 0.6732329084588644
----
Precision:  0.8087120124028273
----
Hamming Loss (%):  2.9374234381380155
----
F1 Score:  0.7452753664819695
----


In [ ]:
df_x = pd.concat([X_title, X_body], axis= 1)
X_train1, X_test1, y_train1, y_test1 = train_test_split(df_x, y, test_size = 0.2, random_state = 0)
df_x = pd.DataFrame(X_test1)
df_x.reset_index(drop=True, inplace=True)

In [ ]:
p = binarizer.inverse_transform(y_pred)
df_p = pd.DataFrame({'Predducted':p})

yTest = binarizer.inverse_transform(y_test)
df_y = pd.DataFrame({'Tags':yTest})

df_p = pd.concat([X_test1, df_y, df_p], axis= 1)
df_p

,Title,Body,Tags,Predducted
0,hibern nosuchfielderror instanc strut 1,new java hibern rail c develop anyway test pro...,"(java,)","(java,)"
1,unit test bad form unit test call unit test,unit test call code testmakeavalidcal code tes...,"(c#,)",()
2,argument unpack use iter item get,use python 2 7 3 p consid dummi class custom a...,"(python,)","(python,)"
3,list paramet caus error,code p pre code public bool somemethod list lt...,"(c#,)","(c#,)"
4,xsd xml document generat c,anyon know xsd file somewher use valid xml doc...,"(c#,)","(c#,)"
...,...,...,...,...
9791,pgp python generat key encrypt decrypt,make program python distribut window user via ...,"(python,)","(python,)"
9792,best practic implement secur rememb,sometim come across certain web develop framew...,"(asp.net,)",()
9793,broadcast receiv check internet connect androi...,hello develop android broadcast receiv check i...,"(android, java)","(android,)"
9794,html code caus caption comment pop mous roll h...,peic html code allow provid comment link user ...,"(html,)","(html,)"


# Example Application

In [ ]:
# Actual Application
df_app = pd.DataFrame({
    'Title':['How to handle or avoid a stack overflow in C++',
             "Python getopt module error NameError: name 'opts' is not defined after importing",
             "get append element but got undefined",
             "I have a problem with pip",
             "Best way to center a <div> on a page vertically and horizontally? "],
    'Body':['In C++ a stack overflow usually leads to an unrecoverable crash of the program. For programs that need to be really robust, this is an unacceptable behaviour, particularly because stack size is limited. A few questions about how to handle the problem. Is there a way to prevent stack overflow by a general technique. (A scalable, robust solution, that includes dealing with external libraries eating a lot of stack, etc.) Is there a way to handle stack overflows in case they occur? Preferably, the stack gets unwound until theres a handler to deal with that kinda issue. There are languages out there, that have threads with expandable stacks. Is something like that possible in C++? Any other helpful comments on the solution of the C++ behaviour would be appreciated.',
            "I'm trying to take in two arguments from the console. The following code seems to have worked on my colleague's computer, so I'm not sure why it is giving me an error when trying to run it on mine. I am on a Mac.import getoptimport sysquestion_id= Nonearg_student = Noneargv = sys.argv[1:]print('test')try:opts, args = getopt.getopt(argv, 'i:s:', ['question_id=','arg_student='])except:print('Error')for opt, arg in opts:if opt in ['-i', '--question_id']:question_id = argelif opt in ['-s', '--arg_student']:arg_student = argprint('Question Number: ' + question_id)print('Student response: ' + arg_student)This is the error I am getting:ErrorTraceback (most recent call last):File '/Users/ailanysmacbook/github/AutomatedEssayGrading/AutomatedEssayGrading/input.py', line 1, in <module>import getoptFile '/Users/ailanysmacbook/github/AutomatedEssayGrading/AutomatedEssayGrading/getopt.py', line 20, in <module>for opt, arg in opts:NameError: name 'opts' is not definedIt seems to be happening right after I try importing it. Do I need to install something? I'm not sure what's missing.",
            "I have append element on my page, and after the element was append i want to send the element value with ajax in javascrypt. But i got response undefinedappend element jquery",
            "I am facimg problems in installing packages from pip",
            "Best way to center a <div> element on a page both vertically and horizontally?I know that margin-left: auto; margin-right: auto; will center on the horizontal, but what is the best way to do it vertically, too?"]
    })

In [ ]:
# preprocessing title and body
df_app['Title'] = df_app['Title'].apply(lambda x: remove_html(x))
df_app['Title'] = df_app['Title'].apply(lambda x: remove_stopwords(x))
df_app['Title'] = df_app['Title'].apply(lambda x: remove_punc(x))
df_app['Title'] = df_app['Title'].apply(lambda x: stem_text(x))

df_app['Body'] = df_app['Body'].apply(lambda x: remove_html(x))
df_app['Body'] = df_app['Body'].apply(lambda x: remove_stopwords(x))
df_app['Body'] = df_app['Body'].apply(lambda x: remove_punc(x))
df_app['Body'] = df_app['Body'].apply(lambda x: stem_text(x))

df_app

,Title,Body
0,handl avoid stack overflow c,c stack overflow usual lead unrecover crash pr...
1,python getopt modul error nameerror name opt d...,tri take two argument consol follow code seem ...
2,get append element got undefin,append element page element append want send e...
3,problem pip,facimg problem instal packag pip
4,best way center div page vertic horizont,best way center div element page vertic horizo...


In [ ]:
x_title = vectorizer_title.transform(df_app['Title'])
x_body = vectorizer_body.transform(df_app['Body'])

P_app = hstack([x_title, x_body])
P_app.shape


(5, 20000)

In [ ]:
py= clf.predict(P_app)
binarizer.inverse_transform(py)

[('c++',), ('python',), ('javascript', 'jquery'), ('python',), ('css', 'html')]